In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd.function import Function
import os
import pickle
import numpy as np
import pandas as pd
import math
from itertools import chain
from sklearn.model_selection import train_test_split
import itertools
import random
import tqdm
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift

from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import nn, einsum
import math
import logging
from functools import partial
from collections import OrderedDict
from sklearn.metrics import classification_report
from torchsummary import summary
import gpustat

In [53]:
!gpustat

/bin/sh: 1: gpustat: not found


In [124]:
seq_len = 3

In [126]:
dat1 = np.load('/project/hikaku_db/data/sleep_SHHS/stages_sig/C4_spec_30_np/spec_1_800_30.npy')
dat2 = np.load('/project/hikaku_db/data/sleep_SHHS/stages_sig/C3_spec_30_np/spec_c3_1_800_30.npy')
print(dat1.shape)
print(dat2.shape)
dat1 = dat1[:790119, :, :]
dat2 = dat2[:790119, :, :]
dat_eeg = np.concatenate((dat1.reshape(-1,seq_len,1,32,30), dat2.reshape(-1,seq_len,1,32,30)), axis=2)
print(dat_eeg.shape)

(790121, 32, 30)
(790121, 32, 30)
(263373, 3, 2, 32, 30)


In [127]:
index = pd.read_csv("/project/hikaku_db/data/sleep_SHHS/stages_sig/ann_delrecords_5class.csv", header=None)
print(index[0 : 790121].apply(pd.value_counts))
label = index[:790119].values
label = label.reshape(-1,seq_len)
print(label.shape)

        0
2  329696
0  221935
4  109008
3  100831
1   28651
(263373, 3)


In [128]:
fixdata = dat_eeg[:,:,:,0:16,:]
mean_p1 = np.mean(dat_eeg[:,:,:,16:20,:], axis = 3)
mean_p2 = np.mean(dat_eeg[:,:,:,20:24,:], axis = 3)
mean_p3 = np.mean(dat_eeg[:,:,:,24:28,:], axis = 3)
mean_p4 = np.mean(dat_eeg[:,:,:,28:32,:], axis = 3)
num_data = len(dat_eeg)
ch = 2
inputeeg = np.concatenate((fixdata,mean_p1.reshape(-1, seq_len, ch, 1, 30),mean_p2.reshape(-1, seq_len, ch, 1, 30),mean_p3.reshape(-1, seq_len, ch, 1, 30),mean_p4.reshape(-1, seq_len, ch, 1, 30)),axis=3)
print(inputeeg.shape)

(263373, 3, 2, 20, 30)


In [131]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data1, label ,transform = None):
        self.transform = transform
        self.data1 = data1
        self.label = label
        self.datanum = len(data1)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        
        out_data1 = torch.tensor(self.data1[idx]).float()
        out_label = torch.tensor(self.label[idx])
        if self.transform:
            out_data1 = self.transform(out_data1)

        return out_data1, out_label

In [132]:
train_eeg, test_eeg, train_label, test_label = train_test_split(inputeeg, label, test_size = 0.1,random_state = 66)
print('train eeg data:',len(train_eeg))
print('test eeg data:',len(test_eeg))

train_data_set = Mydatasets(data = train_eeg, label = train_label)
test_data_set = Mydatasets(data = test_eeg, label = test_label)

train_dataloader = torch.utils.data.DataLoader(train_data_set, batch_size = 16, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data_set, batch_size = 16, shuffle=False)

train eeg data: 237035
test eeg data: 26338


In [148]:
##Seq_VTransformer model

class PreNorm(nn.Module): 
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, mlp_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, mlp_dim),       #output_of_attention_dim to mlp_dim
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, dim),      #mlp_dim to output_of_attention_dim(==input_of_attention_dim)
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, dropout, heads = 4, dim_head = 8):
        super().__init__()
        inner_dim = dim_head * heads       ##qkv_dim(inner_dim) = head_num * head_dim     32 = 4*8
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)   ##embedding_dim to qkv_dim * 3

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale #dots for attention-value(scaled) of Q -> K
        
        attn = self.attend(dots)             #atte for Softmax(attention-value)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)        #out for Z = attn * V
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)             #out_dim(==inner_dim==Z_dim) to embedding_dim

class Transformer(nn.Module):           ##Register the blocks into whole network
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x     #Residuals cut-in
            x = ff(x) + x
        return x

class ViT(nn.Module):
    def __init__(self,image_size, time_size, fre_size, dim, channels , dim_head,depth, heads, mlp_dim, dropout, pool = 'cls', emb_dropout = 0.):
        super().__init__()
        assert image_size == 30  ##Time dimensions must equal to 30s
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'
        num_patches = 150       #30*5(5EEG)
        patch_dim = channels * time_size * fre_size    #EEG_patch_dim:2*1*4  
    
        self.eeg_to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p3) (w p4) -> b (h w) (p3 p4 c)', p3 = fre_size, p4 = time_size),
            nn.Linear(patch_dim, dim),       ##eeg_patch_dim(2*1*4) to embed_dim
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim)) ##Generate the pos value'
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim)) ##Generate the class value'
        self.dropout = nn.Dropout(emb_dropout)
        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout) 
        #self.seq_transformer = Transformer(dim*2, depth, heads, dim_head, mlp_dim, dropout)# ill defined,useless
        
        self.pool = pool
        self.dim = dim
        self.to_latent = nn.Identity()
        


    def forward(self, input_eeg):
        dim = self.dim
        batch = input_eeg.shape[0]
        cls_token_seq = torch.empty(batch,0,dim).to(DEVICE)
        for seq in range(seq_len):
            eeg = input_eeg[:,seq,:,:,:]
            EEG = self.eeg_to_patch_embedding(eeg)
            x = EEG            
            b, n, _ = x.shape
            cls_tokens = repeat(self.cls_token, '(      ) n d -> b n d', b = b)
            x = torch.cat((cls_tokens, x), dim=1)
            x += self.pos_embedding[:, :(n + 1)]
            x = self.dropout(x)
            x = self.transformer(x)
            x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]
            x = self.to_latent(x)
            x = x.reshape(batch,-1,dim) 
            cls_token_seq = torch.cat((cls_token_seq, x), dim=1)            
        return cls_token_seq   

class seq_ViT(nn.Module):
    def __init__(self,num_classes, seq_len, dim, dim_head, depth, heads, mlp_dim, dropout, pool = 'cls', emb_dropout = 0.):
        super().__init__()
        num_patches = seq_len       
        seq_dim = dim 
        embedding_dim = dim*2
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'
        
        self.seq_to_patch_embedding = nn.Sequential(
            nn.Linear(seq_dim, embedding_dim),      
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embedding_dim)) ##Generate the pos value'
        self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim)) ##Generate the class value'
        self.dropout = nn.Dropout(emb_dropout)
        self.transformer = Transformer(embedding_dim, depth, heads, dim_head, mlp_dim, dropout) 
        
        self.pool = pool
        self.dim = dim
        self.to_latent = nn.Identity()
        
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embedding_dim),
            nn.Linear(embedding_dim, num_classes),
            nn.Dropout(dropout)
        )

    def forward(self, input_seq):
        seq = self.seq_to_patch_embedding(input_seq)         
        b, seq_len, n = seq.shape
        cls_tokens = repeat(self.cls_token, '(      ) n d -> b n d', b = b)
        seq = torch.cat((cls_tokens, seq), dim=1)
        seq += self.pos_embedding[:, :(n + 1)]
        seq = self.dropout(seq)
        seq = self.transformer(seq)
        seq = seq.mean(dim = 1) if self.pool == 'mean' else seq[:, 0]
        mlp_out = self.mlp_head(seq)         
        return F.log_softmax(mlp_out, dim = -1)   
    
class Seq_VTrans(nn.Module):
    def __init__(self, *, seq_len,num_classes,image_size, time_size, fre_size, dim, channels , dim_head,depth, heads, mlp_dim, dropout, pool = 'cls', emb_dropout = 0.): 
        super().__init__()
        self.trans = ViT(image_size, time_size, fre_size, dim, channels, dim_head, depth, heads, mlp_dim, dropout, pool, emb_dropout)
        self.seq_trans = seq_ViT(num_classes, seq_len, dim, dim_head, depth, heads, mlp_dim, dropout, pool, emb_dropout)
        
    def forward(self,input_eeg):
        cls_token_seq = self.trans(input_eeg)
        out = self.seq_trans(cls_token_seq)
        return out

In [149]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

Seq_VTransmodel = Seq_VTrans(
    image_size = 30, 
    time_size = 1, 
    fre_size = 4, 
    num_classes = 5,
    channels = 2,
    seq_len = 3,   
    depth = 8,   
    heads = 4, 
    dim = 32,     #32
    dim_head = 32,    #32
    mlp_dim = 128,     #128
    dropout = 0.0
).to(DEVICE)

optimizer = torch.optim.AdamW(Seq_VTransmodel.parameters(), lr = 1e-6)
Classifier_loss = torch.nn.CrossEntropyLoss()

#Seq_VTransmodel.load_state_dict(torch.load('/project/hikaku_db/ziwei/Model_VT_VT/Model_VT_VT_state_diff_shape_1'))

cuda:0


In [ ]:
EPOCH = 100
loss_list=[]
los_min=10**10
val_loss_list=[]
ac_list=[]


for epoch in tqdm.tqdm(range(EPOCH)):
    running_loss = 0.0
    count=0
    for _, (inputs, labels) in enumerate(train_dataloader, 0):
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        label_one_hot = F.one_hot(labels, 5).float()
        cls_out = Seq_VTransmodel(inputs)
        count = count+1      
        loss = Classifier_loss(cls_out, label_one_hot[:,1,:])
        loss.backward()
        optimizer.step() 
        running_loss += loss.item()
    loss_loss=running_loss/count
    loss_list.append(loss_loss)
    print('epoch',epoch+1,':finished')
    print('train_loss:',loss_loss)

    #####test aphase 
    with torch.no_grad():
        count=0
        running_loss=0.0
        pre=list()
        lab=list()
        for _, (inputs, labels) in enumerate(test_dataloader, 0):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            label_one_hot = F.one_hot(labels, 5).float()
            cls_out = Seq_VTransmodel(inputs)
            count = count+1      
            loss = Classifier_loss(cls_out, label_one_hot[:,1,:]) 
            running_loss += loss.item()
            predicted = cls_out.max(1)[1].to('cpu') 
            groudT = label_one_hot[:,1,:].max(1)[1].to('cpu')
            predicted = predicted.tolist()
            groudT = groudT.tolist()
            pre.append(predicted)
            lab.append(groudT)
        loss_loss = running_loss/count
        val_loss_list.append(loss_loss)
        pre=sum(pre,[])
        lab=sum(lab,[])
        print('val_loss:',loss_loss)
        cl = classification_report(lab, pre,output_dict=True)
        print(cl)
        ac_list.append(cl['accuracy'])
        
        #torch.save(Seq_VTransmodel.state_dict(),'/project/hikaku_db/ziwei/Model_VT_VT/Model_VT_VT_state_3')

In [ ]:
plt.plot(loss_list[1:])
plt.plot(val_loss_list[1:])
#plt.savefig('/project/hikaku_db/ziwei/Model_VT_VT/loss_plt_diff_shape_1.jpg')

In [ ]:
plt.plot(ac_list)
#plt.savefig('/project/hikaku_db/ziwei/Model_VT_VT/ac_plt_diff_shape_1.jpg')